# PostgreSQL: how to access from JupyterHub

[PostgreSQL](https://www.postgresql.org/) is a powerful, open source object-relational database system with over 30 years of active development that has earned it a strong reputation for reliability, feature robustness, and performance.

We have a cloud-based installation of PostgreSQL database that includes [flights demo database](https://postgrespro.com/education/demodb) in read-only format for you to learn how to use PostgreSQL in your tasks.

## Import libraries and set access parameters

In [ ]:
import os
import json
import psycopg2
from psycopg2.extras import DictCursor

Set host to get to database. PostgreSQL has no external IP address, so it is available only from JupyterHub notebooks:

In [ ]:
POSTGRESQL_HOST = '10.129.0.25'

[Psycopg](https://www.psycopg.org/docs/index.html) is installed in the JupyterHub environment as a PostgreSQL database adapter for Python. So we are going to use this library for our demo access.

## Access option #1 (close connection and cursor manually)

In order to connect to PostgreSQL database a [cursor](https://www.psycopg.org/docs/cursor.html) should be created with parameters like database's host, username and password.

In [ ]:
!env | grep POST

In [ ]:
conn = psycopg2.connect(
    dbname='demo', 
    user=os.environ['POSTGRESQL_USER'],
    password=os.environ['POSTGRESQL_PASSWORD'], 
    host=POSTGRESQL_HOST
)
cur = conn.cursor()

__NOTE:__ Username (or login) and password are stored in environment variables `POSTGRESQL_USER`, `POSTGRESQL_PASSWORD` for safety and are available through `os` library as `os.environ['<ENV_VARIABLE_NAME>']`.

Now create SQL query:

In [ ]:
query = 'SELECT * FROM seats LIMIT 5'

...and run it:

In [ ]:
cur.execute(query)
records = cur.fetchall()
cur.close()
conn.close()

Data recieved is now in `records` variable:

In [ ]:
records

## Access option #2 (use 'with' context manager)

In [ ]:
with psycopg2.connect(
    dbname='demo', 
    user=os.environ['POSTGRESQL_USER'],
    password=os.environ['POSTGRESQL_PASSWORD'], 
    host=POSTGRESQL_HOST
) as conn:
    with conn.cursor() as cur:
        cur.execute('SELECT * FROM seats LIMIT 5')
        records = cur.fetchall()

In [ ]:
records

## Multiply queries

A list of SQL queries in one connection:

In [ ]:
queries = {
    '___DATABASES___': 'SELECT * FROM pg_database',
    '___TABLES___': "SELECT relname FROM pg_class WHERE relkind='r' AND relname !~ '^(pg_|sql_)';",
    '\n___COLUMNS___': "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'seats';"
}

In [ ]:
with psycopg2.connect(
    dbname='demo', 
    user=os.environ['POSTGRESQL_USER'],
    password=os.environ['POSTGRESQL_PASSWORD'], 
    host=POSTGRESQL_HOST
) as conn:
    for name, query in queries.items():
        print('\n', name)
        with conn.cursor() as cur:
            cur.execute(query)
            for row in cur.fetchall():
                print(row)

## SQL for IPython

The [IPython SQL](https://github.com/catherinedevlin/ipython-sql) framework allows to use [magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) like `%sql` (or `%%sql`) for SQL queries. We can connect to a database, then issue SQL commands within Jupyter.

Let's install library first:

In [ ]:
!pip install ipython-sql

Enable `sql` in Jupyter notebook cells:

In [ ]:
%load_ext sql

Connection data string to be used later:

In [ ]:
CONNECT_DATA = 'postgresql://{}:{}@{}/{}'.format(
    os.environ['POSTGRESQL_USER'],
    os.environ['POSTGRESQL_PASSWORD'],
    POSTGRESQL_HOST,
    'demo'
)

Let's look at all databases in PostgreSQL. SQL query can be done after the connection with `%%sql` magic command:

In [ ]:
%%sql $CONNECT_DATA
    SELECT * FROM pg_database

Connect string can be omitted (just use `%sql` NOT the `%%sql`) if connection is done earlier. Now will get all tables in `demo` database:

In [ ]:
%sql SELECT tablename AS table FROM pg_tables WHERE tablename !~ '^(pg_|sql_)'